In [ ]:
!pip install transformers
!pip install transformers torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-base-mnli')
model = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-base-mnli')